In [1]:
pip install requests beautifulsoup4 selenium

  Using cached websocket_client-1.8.0-py3-none-any.whl.metadata (8.0 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached pycparser-2.22-py3-none-any.whl.metadata (943 bytes)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
   ---------------------------------------- 0.0/9.4 MB ? eta -:--:--
    --------------------------------------- 0.1/9.4 MB 2.4 MB/s eta 0:00:04
   - -------------------------------------- 0.3/9.4 MB 2.9 MB/s eta 0:00:04
   -- ------------------------------------- 0.6/9.4 MB 4.2 MB/s eta 0:00:03
   ------ --------------------------------- 1.5/9.4 MB 7.7 MB/s eta 0:00:02
   ---------- ----------------------------- 2.6/9.4 MB 11.8 MB/s eta 0:00:01
   ------------ --------------------------- 3.0/9.4 MB 10.7 MB/s eta 0:00:01
   --------------- ------------------------ 3.6/9.4 MB 11.3 MB/s eta 0:00:01
   ---------------- ----------------------- 3.8/9.4 MB 10.5 MB/s eta 0:00:01
   ---------------------- ----------------- 5.2/9.4 M


[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import requests
from bs4 import BeautifulSoup

In [20]:
# 구글 플레이스토어 URL
url = "https://play.google.com/store/apps/details?id=com.kakao.talk"

# HTTP GET 요청
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}
response = requests.get(url, headers=headers)

In [21]:
if response.status_code == 200:
    print("Success")
else:
    print(f"Failed to retrieve the page. Status code: {response.status_code}")

Success


In [22]:
# 요청이 성공했는지 확인
if response.status_code == 200:
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    
    # 앱 제목 가져오기
    app_title = soup.find('h1', {'class': 'Fd93Bb'}).text if soup.find('h1', {'class': 'Fd93Bb'}) else 'Title not found'
    print(f"App Title: {app_title}")

    # 앱 설명 가져오기
    app_description_meta = soup.find('meta', {'itemprop': 'description'})
    app_description = app_description_meta['content'] if app_description_meta else 'Description not found'
    print(f"App Description: {app_description}")

    # 앱 카테고리 가져오기
    app_category_button = soup.find('button', {'data-idom-class': 'Rj2Mlf OLiIxf PDpWxe P62QJc LQeN7 LMoCf'})
    app_category = app_category_button.find('span', {'jsname': 'V67aGc'}).text if app_category_button else 'Category not found'
    print(f"App Category: {app_category}")

else:
    print(f"Failed to retrieve the page. Status code: {response.status_code}")

App Title: KakaoTalk : Messenger
App Description: KakaoTalk - fast, fun, and reliable messenger for all
App Category: #1 popular communication


In [16]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import quote

def get_search_url(app_name):
    base_url = "https://play.google.com/store/search?q="
    query = quote(app_name)
    return f"{base_url}{query}&c=apps"

def get_app_urls(search_query, num_results=1):
    search_url = get_search_url(search_query)
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    response = requests.get(search_url, headers=headers)
    
    if response.status_code == 200:
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')
        
        app_urls = []
        for a in soup.find_all('a', {'class': 'Si6A0c Gy4nib'}, limit=num_results):
            app_url = 'https://play.google.com' + a['href']
            app_urls.append(app_url)
        
        return app_urls
    else:
        print(f"Failed to retrieve the search page. Status code: {response.status_code}")
        return []

def get_app_info(app_url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    response = requests.get(app_url, headers=headers)
    
    if response.status_code == 200:
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')
        
        # 앱 제목 가져오기
        app_title = soup.find('h1', {'class': 'Fd93Bb'}).text if soup.find('h1', {'class': 'Fd93Bb'}) else 'Title not found'
        
        # 앱 설명 가져오기
        app_description_meta = soup.find('meta', {'itemprop': 'description'})
        app_description = app_description_meta['content'] if app_description_meta else 'Description not found'
        
        # 앱 카테고리 가져오기
        app_category_button = soup.find('button', {'data-idom-class': 'Rj2Mlf OLiIxf PDpWxe P62QJc LQeN7 LMoCf'})
        app_category = app_category_button.find('span', {'jsname': 'V67aGc'}).text if app_category_button else 'Category not found'
        # print(f"App Category: {app_category}")
        
        return {
            'title': app_title,
            'description': app_description,
            'category': app_category
        }
    else:
        print(f"Failed to retrieve the app page. Status code: {response.status_code}")
        return {}

# 앱 이름 리스트
app_names = ["카카오톡", "파파고", "카카오맵", "네이버", "쿠팡"]

# 각 앱의 URL 수집 및 정보 크롤링
for app_name in app_names:
    app_urls = get_app_urls(app_name)
    if app_urls:
        app_info = get_app_info(app_urls[0])
        print(app_info)
    else:
        print(f"No URL found for app: {app_name}")


{'title': 'KakaoTalk : Messenger', 'description': 'KakaoTalk - fast, fun, and reliable messenger for all', 'category': '#1 popular communication'}
{'title': 'Naver Papago - AI Translator', 'description': 'Smart AI Translator, dreaming of a world communicating without language barriers', 'category': '#2 popular tools'}
{'title': '카카오내비 - 주차,발렛,전기차충전,세차,보험,중고차', 'description': 'The new evolution of navigation, KakaoNavi', 'category': '#7 popular maps & navigation'}
{'title': 'NAVER Map, Navigation', 'description': "South Korea's GPS navigation Get started right away", 'category': '#2 popular maps & navigation'}
{'title': 'Coupang Play', 'description': "Enjoy COUPANG PLAY's Unlimited Movies&TV shows only for Coupang WOW members.", 'category': '#2 popular entertainment'}


In [13]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import quote

def get_search_url(app_name):
    base_url = "https://play.google.com/store/search?q="
    query = quote(app_name)
    return f"{base_url}{query}&c=apps"

def get_app_urls(search_query, num_results=1):
    search_url = get_search_url(search_query)
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    response = requests.get(search_url, headers=headers)
    
    if response.status_code == 200:
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')
        
        app_urls = []
        for a in soup.find_all('a', {'class': 'Si6A0c Gy4nib'}, limit=num_results):
            app_url = 'https://play.google.com' + a['href']
            app_urls.append(app_url)
        
        return app_urls
    else:
        print(f"Failed to retrieve the search page. Status code: {response.status_code}")
        return []

def get_app_info(app_url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    response = requests.get(app_url, headers=headers)
    
    if response.status_code == 200:
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')
        
        # 앱 제목 가져오기
        app_title = soup.find('h1', {'class': 'Fd93Bb'}).text if soup.find('h1', {'class': 'Fd93Bb'}) else 'Title not found'
        
        # 앱 설명 가져오기
        app_description_meta = soup.find('meta', {'itemprop': 'description'})
        app_description = app_description_meta['content'] if app_description_meta else 'Description not found'
        
        # 앱 카테고리 가져오기
        app_category_button = soup.find('a', {'itemprop': 'genre'})
        app_category = app_category_button.text if app_category_button else 'Category not found'
        
        return {
            'url': app_url,
            'title': app_title,
            'description': app_description,
            'category': app_category
        }
    else:
        print(f"Failed to retrieve the app page. Status code: {response.status_code}")
        return {}

# 앱 이름 리스트
app_names = ["카카오톡", "파파고", "카카오맵", "네이버", "쿠팡"]

# 각 앱의 URL 수집 및 정보 크롤링
for app_name in app_names:
    app_urls = get_app_urls(app_name)
    if app_urls:
        app_info = get_app_info(app_urls[0])
        print(app_info)
    else:
        print(f"No URL found for app: {app_name}")


{'url': 'https://play.google.com/store/apps/details?id=com.kakao.talk', 'title': 'KakaoTalk : Messenger', 'description': 'KakaoTalk - fast, fun, and reliable messenger for all', 'category': 'Category not found'}
{'url': 'https://play.google.com/store/apps/details?id=com.naver.labs.translator', 'title': 'Naver Papago - AI Translator', 'description': 'Smart AI Translator, dreaming of a world communicating without language barriers', 'category': 'Category not found'}
{'url': 'https://play.google.com/store/apps/details?id=com.locnall.KimGiSa', 'title': '카카오내비 - 주차,발렛,전기차충전,세차,보험,중고차', 'description': 'The new evolution of navigation, KakaoNavi', 'category': 'Category not found'}
{'url': 'https://play.google.com/store/apps/details?id=com.nhn.android.nmap', 'title': 'NAVER Map, Navigation', 'description': "South Korea's GPS navigation Get started right away", 'category': 'Category not found'}
{'url': 'https://play.google.com/store/apps/details?id=com.coupang.mobile.play', 'title': 'Coupang Pl

In [8]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import quote

def get_search_url(app_name):
    base_url = "https://play.google.com/store/search?q="
    query = quote(app_name)
    return f"{base_url}{query}&c=apps"

def get_app_urls(search_query, expected_name, num_results=5):
    search_url = get_search_url(search_query)
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    response = requests.get(search_url, headers=headers)
    
    if response.status_code == 200:
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')
        
        app_urls = []
        for a in soup.find_all('a', href=True, limit=num_results):
            if '/store/apps/details?id=' in a['href']:
                app_url = 'https://play.google.com' + a['href']
                app_title_div = a.find('div', {'class': 'WsMG1c nnK0zc'})
                if app_title_div:
                    app_title = app_title_div.text
                    if expected_name.lower() in app_title.lower():
                        app_urls.append(app_url)
        
        return app_urls
    else:
        print(f"Failed to retrieve the search page. Status code: {response.status_code}")
        return []

def get_app_info(app_url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    response = requests.get(app_url, headers=headers)
    
    if response.status_code == 200:
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')
        
        # 앱 제목 가져오기
        app_title = soup.find('h1', {'class': 'Fd93Bb'}).text if soup.find('h1', {'class': 'Fd93Bb'}) else 'Title not found'
        
        # 앱 설명 가져오기
        app_description_meta = soup.find('meta', {'itemprop': 'description'})
        app_description = app_description_meta['content'] if app_description_meta else 'Description not found'
        
        # 앱 카테고리 가져오기
        app_category_button = soup.find('a', {'itemprop': 'genre'})
        app_category = app_category_button.text if app_category_button else 'Category not found'
        
        return {
            'url': app_url,
            'title': app_title,
            'description': app_description,
            'category': app_category
        }
    else:
        print(f"Failed to retrieve the app page. Status code: {response.status_code}")
        return {}

# 앱 이름 리스트
app_names = {
    "카카오톡": "카카오톡 KakaoTalk",
    "파파고": "Papago",
    "카카오맵": "KakaoMap",
    "네이버": "NAVER",
    "쿠팡": "Coupang"
}

# 각 앱의 URL 수집 및 정보 크롤링
for app_name, expected_name in app_names.items():
    app_urls = get_app_urls(app_name, expected_name)
    if app_urls:
        app_info = get_app_info(app_urls[0])
        print(app_info)
    else:
        print(f"No URL found for app: {app_name}")


No URL found for app: 카카오톡
No URL found for app: 파파고
No URL found for app: 카카오맵
No URL found for app: 네이버
No URL found for app: 쿠팡


In [9]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import quote

def get_search_url(app_name):
    base_url = "https://play.google.com/store/search?q="
    query = quote(app_name)
    return f"{base_url}{query}&c=apps"

def get_app_urls(search_query, expected_name, num_results=5):
    search_url = get_search_url(search_query)
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    response = requests.get(search_url, headers=headers)
    
    if response.status_code == 200:
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')
        
        app_urls = []
        for a in soup.find_all('a', href=True, limit=num_results):
            if '/store/apps/details?id=' in a['href']:
                app_url = 'https://play.google.com' + a['href']
                app_title_div = a.find('div', {'class': 'WsMG1c nnK0zc'})
                if app_title_div:
                    app_title = app_title_div.text
                    if expected_name.lower() in app_title.lower():
                        app_urls.append(app_url)
        
        return app_urls
    else:
        print(f"Failed to retrieve the search page. Status code: {response.status_code}")
        return []

def get_app_info(app_url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    response = requests.get(app_url, headers=headers)
    
    if response.status_code == 200:
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')
        
        # 앱 제목 가져오기
        app_title = soup.find('h1', {'class': 'Fd93Bb'}).text if soup.find('h1', {'class': 'Fd93Bb'}) else 'Title not found'
        
        # 앱 설명 가져오기
        app_description_meta = soup.find('meta', {'itemprop': 'description'})
        app_description = app_description_meta['content'] if app_description_meta else 'Description not found'
        
        # 앱 카테고리 가져오기
        app_category_button = soup.find('a', {'itemprop': 'genre'})
        app_category = app_category_button.text if app_category_button else 'Category not found'
        
        return {
            'url': app_url,
            'title': app_title,
            'description': app_description,
            'category': app_category
        }
    else:
        print(f"Failed to retrieve the app page. Status code: {response.status_code}")
        return {}

# 앱 이름 리스트
app_names = {
    "카카오톡": "카카오톡 KakaoTalk",
    "파파고": "Papago",
    "카카오맵": "KakaoMap",
    "네이버": "NAVER",
    "쿠팡": "Coupang"
}

# 각 앱의 URL 수집 및 정보 크롤링
for app_name, expected_name in app_names.items():
    app_urls = get_app_urls(app_name, expected_name)
    if app_urls:
        app_info = get_app_info(app_urls[0])
        print(app_info)
    else:
        print(f"No URL found for app: {app_name}")


No URL found for app: 카카오톡
No URL found for app: 파파고
No URL found for app: 카카오맵
No URL found for app: 네이버
No URL found for app: 쿠팡


In [10]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import quote

def get_search_url(app_name):
    base_url = "https://play.google.com/store/search?q="
    query = quote(app_name)
    return f"{base_url}{query}&c=apps"

def get_app_urls(search_query, expected_name, num_results=5):
    search_url = get_search_url(search_query)
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    response = requests.get(search_url, headers=headers)
    
    if response.status_code == 200:
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')
        
        app_urls = []
        for a in soup.find_all('a', href=True, limit=num_results):
            if '/store/apps/details?id=' in a['href']:
                app_url = 'https://play.google.com' + a['href']
                app_title_div = a.find('span', {'class': 'DdYX5'})
                if app_title_div:
                    app_title = app_title_div.text
                    if expected_name.lower() in app_title.lower():
                        app_urls.append(app_url)
        
        return app_urls
    else:
        print(f"Failed to retrieve the search page. Status code: {response.status_code}")
        return []

def get_app_info(app_url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    response = requests.get(app_url, headers=headers)
    
    if response.status_code == 200:
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')
        
        # 앱 제목 가져오기
        app_title = soup.find('h1', {'class': 'Fd93Bb'}).text if soup.find('h1', {'class': 'Fd93Bb'}) else 'Title not found'
        
        # 앱 설명 가져오기
        app_description_meta = soup.find('meta', {'itemprop': 'description'})
        app_description = app_description_meta['content'] if app_description_meta else 'Description not found'
        
        # 앱 카테고리 가져오기
        app_category_button = soup.find('a', {'itemprop': 'genre'})
        app_category = app_category_button.text if app_category_button else 'Category not found'
        
        return {
            'url': app_url,
            'title': app_title,
            'description': app_description,
            'category': app_category
        }
    else:
        print(f"Failed to retrieve the app page. Status code: {response.status_code}")
        return {}

# 앱 이름 리스트
app_names = {
    "카카오톡": "카카오톡 KakaoTalk",
    "파파고": "Papago",
    "카카오맵": "KakaoMap",
    "네이버": "NAVER",
    "쿠팡": "Coupang"
}

# 각 앱의 URL 수집 및 정보 크롤링
for app_name, expected_name in app_names.items():
    app_urls = get_app_urls(app_name, expected_name)
    if app_urls:
        app_info = get_app_info(app_urls[0])
        print(app_info)
    else:
        print(f"No URL found for app: {app_name}")


No URL found for app: 카카오톡
No URL found for app: 파파고
No URL found for app: 카카오맵
No URL found for app: 네이버
No URL found for app: 쿠팡


In [12]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import quote

def get_search_url(app_name):
    base_url = "https://play.google.com/store/search?q="
    query = quote(app_name)
    return f"{base_url}{query}&c=apps"

def get_app_urls(search_query, expected_name, num_results=5):
    search_url = get_search_url(search_query)
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    response = requests.get(search_url, headers=headers)
    
    if response.status_code == 200:
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')
        
        app_urls = []
        for a in soup.find_all('a', href=True, limit=num_results):
            if '/store/apps/details?id=' in a['href']:
                app_url = 'https://play.google.com' + a['href']
                app_title = a.find('div', {'class': 'WsMG1c nnK0zc'}).text
                if expected_name.lower() in app_title.lower():
                    app_urls.append(app_url)
        
        return app_urls
    else:
        print(f"Failed to retrieve the search page. Status code: {response.status_code}")
        return []

def get_app_info(app_url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    response = requests.get(app_url, headers=headers)
    
    if response.status_code == 200:
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')
        
        # 앱 제목 가져오기
        app_title = soup.find('h1', {'class': 'Fd93Bb'}).text if soup.find('h1', {'class': 'Fd93Bb'}) else 'Title not found'
        
        # 앱 설명 가져오기
        app_description_meta = soup.find('meta', {'itemprop': 'description'})
        app_description = app_description_meta['content'] if app_description_meta else 'Description not found'
        
        # 앱 카테고리 가져오기
        app_category_button = soup.find('a', {'itemprop': 'genre'})
        app_category = app_category_button.text if app_category_button else 'Category not found'
        
        return {
            'url': app_url,
            'title': app_title,
            'description': app_description,
            'category': app_category
        }
    else:
        print(f"Failed to retrieve the app page. Status code: {response.status_code}")
        return {}

# 앱 이름과 예상된 정확한 제목 매핑 리스트
app_names = {
    "카카오톡": "카카오톡",
    "파파고": "파파고",
    "카카오맵": "카카오맵",
    "네이버": "네이버",
    "쿠팡": "쿠팡"
}

# 각 앱의 URL 수집 및 정보 크롤링
for app_name, expected_name in app_names.items():
    app_urls = get_app_urls(app_name, expected_name)
    if app_urls:
        app_info = get_app_info(app_urls[0])
        print(app_info)
    else:
        print(f"No URL found for app: {app_name}")


No URL found for app: 카카오톡
No URL found for app: 파파고
No URL found for app: 카카오맵
No URL found for app: 네이버
No URL found for app: 쿠팡


In [15]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import quote

def get_search_url(app_name):
    base_url = "https://play.google.com/store/search?q="
    query = quote(app_name)
    return f"{base_url}{query}&c=apps"

def get_first_app_url(search_query):
    search_url = get_search_url(search_query)
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    response = requests.get(search_url, headers=headers)
    
    if response.status_code == 200:
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')
        
        first_result = soup.find('a', {'class': 'Si6A0c Gy4nib'})
        if first_result:
            app_url = 'https://play.google.com' + first_result['href']
            return app_url
        else:
            print(f"No app found for search query: {search_query}")
            return None
    else:
        print(f"Failed to retrieve the search page. Status code: {response.status_code}")
        return None

def get_app_info(app_url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    response = requests.get(app_url, headers=headers)
    
    if response.status_code == 200:
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')
        
        # 앱 제목 가져오기
        app_title = soup.find('h1', {'class': 'Fd93Bb'}).text if soup.find('h1', {'class': 'Fd93Bb'}) else 'Title not found'
        
        # 앱 설명 가져오기
        app_description_meta = soup.find('meta', {'itemprop': 'description'})
        app_description = app_description_meta['content'] if app_description_meta else 'Description not found'
        
        # 앱 카테고리 가져오기
        app_category_button = soup.find('a', {'itemprop': 'genre'})
        app_category = app_category_button.text if app_category_button else 'Category not found'
        
        return {
            'url': app_url,
            'title': app_title,
            'description': app_description,
            'category': app_category
        }
    else:
        print(f"Failed to retrieve the app page. Status code: {response.status_code}")
        return None

# 앱 이름 리스트
app_names = ["카카오톡", "파파고", "카카오맵", "네이버", "쿠팡"]

# 각 앱의 URL 수집 및 정보 크롤링
for app_name in app_names:
    app_url = get_first_app_url(app_name)
    if app_url:
        app_info = get_app_info(app_url)
        print(app_info)
    else:
        print(f"No URL found for app: {app_name}")


{'url': 'https://play.google.com/store/apps/details?id=com.kakao.talk', 'title': 'KakaoTalk : Messenger', 'description': 'KakaoTalk - fast, fun, and reliable messenger for all', 'category': 'Category not found'}
{'url': 'https://play.google.com/store/apps/details?id=com.naver.labs.translator', 'title': 'Naver Papago - AI Translator', 'description': 'Smart AI Translator, dreaming of a world communicating without language barriers', 'category': 'Category not found'}
{'url': 'https://play.google.com/store/apps/details?id=com.locnall.KimGiSa', 'title': '카카오내비 - 주차,발렛,전기차충전,세차,보험,중고차', 'description': 'The new evolution of navigation, KakaoNavi', 'category': 'Category not found'}
{'url': 'https://play.google.com/store/apps/details?id=com.nhn.android.nmap', 'title': 'NAVER Map, Navigation', 'description': "South Korea's GPS navigation Get started right away", 'category': 'Category not found'}
{'url': 'https://play.google.com/store/apps/details?id=com.coupang.mobile.play', 'title': 'Coupang Pl

In [16]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import quote

def get_search_url(app_name):
    base_url = "https://play.google.com/store/search?q="
    query = quote(app_name)
    return f"{base_url}{query}&c=apps"

def get_best_match_app_url(search_query, expected_name, num_results=5):
    search_url = get_search_url(search_query)
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    response = requests.get(search_url, headers=headers)
    
    if response.status_code == 200:
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')
        
        app_urls = []
        app_titles = []
        for a in soup.find_all('a', {'class': 'Si6A0c Gy4nib'}, limit=num_results):
            app_url = 'https://play.google.com' + a['href']
            app_title = a.find('div', {'class': 'WsMG1c nnK0zc'}).text
            app_urls.append(app_url)
            app_titles.append(app_title)
        
        # Find the best match
        best_match_url = None
        for i in range(len(app_titles)):
            if expected_name.lower() in app_titles[i].lower():
                best_match_url = app_urls[i]
                break
        
        return best_match_url
    else:
        print(f"Failed to retrieve the search page. Status code: {response.status_code}")
        return None

def get_app_info(app_url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    response = requests.get(app_url, headers=headers)
    
    if response.status_code == 200:
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')
        
        # 앱 제목 가져오기
        app_title = soup.find('h1', {'class': 'Fd93Bb'}).text if soup.find('h1', {'class': 'Fd93Bb'}) else 'Title not found'
        
        # 앱 설명 가져오기
        app_description_meta = soup.find('meta', {'itemprop': 'description'})
        app_description = app_description_meta['content'] if app_description_meta else 'Description not found'
        
        # 앱 카테고리 가져오기
        app_category_button = soup.find('a', {'itemprop': 'genre'})
        app_category = app_category_button.text if app_category_button else 'Category not found'
        
        return {
            'url': app_url,
            'title': app_title,
            'description': app_description,
            'category': app_category
        }
    else:
        print(f"Failed to retrieve the app page. Status code: {response.status_code}")
        return None

# 앱 이름과 예상된 정확한 제목 매핑 리스트
app_names = {
    "카카오톡": "KakaoTalk",
    "파파고": "Papago",
    "카카오맵": "KakaoMap",
    "네이버": "NAVER",
    "쿠팡": "Coupang"
}

# 각 앱의 URL 수집 및 정보 크롤링
for app_name, expected_name in app_names.items():
    app_url = get_best_match_app_url(app_name, expected_name)
    if app_url:
        app_info = get_app_info(app_url)
        print(app_info)
    else:
        print(f"No URL found for app: {app_name}")


AttributeError: 'NoneType' object has no attribute 'text'

In [19]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import quote

def get_search_url(app_name):
    base_url = "https://play.google.com/store/search?q="
    query = quote(app_name)
    return f"{base_url}{query}&c=apps"

def get_best_match_app_url(search_query, expected_name, num_results=5):
    search_url = get_search_url(search_query)
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    response = requests.get(search_url, headers=headers)
    
    if response.status_code == 200:
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')
        
        app_urls = []
        app_titles = []
        for a in soup.find_all('a', {'class': 'Si6A0c Gy4nib'}, limit=num_results):
            app_url = 'https://play.google.com' + a['href']
            app_title_div = a.find('div', {'class': 'WsMG1c nnK0zc'})
            if app_title_div:
                app_title = app_title_div.text
            else:
                app_title_span = a.find('span', {'class': 'DdYX5'})
                app_title = app_title_span.text if app_title_span else 'Title not found'
            app_urls.append(app_url)
            app_titles.append(app_title)
        
        # Find the best match
        best_match_url = None
        for i in range(len(app_titles)):
            if expected_name.lower() in app_titles[i].lower():
                best_match_url = app_urls[i]
                break
        
        return best_match_url
    else:
        print(f"Failed to retrieve the search page. Status code: {response.status_code}")
        return None

def get_app_info(app_url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    response = requests.get(app_url, headers=headers)
    
    if response.status_code == 200:
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')
        
        # 앱 제목 가져오기
        app_title = soup.find('h1', {'class': 'Fd93Bb'}).text if soup.find('h1', {'class': 'Fd93Bb'}) else 'Title not found'
        
        # 앱 설명 가져오기
        app_description_meta = soup.find('meta', {'itemprop': 'description'})
        app_description = app_description_meta['content'] if app_description_meta else 'Description not found'
        
        # 앱 카테고리 가져오기
        app_category_button = soup.find('button', {'data-idom-class': 'Rj2Mlf OLiIxf PDpWxe P62QJc LQeN7 LMoCf'})
        app_category = app_category_button.find('span', {'jsname': 'V67aGc'}).text if app_category_button else 'Category not found'
        
        return {
            'url': app_url,
            'title': app_title,
            'description': app_description,
            'category': app_category
        }
    else:
        print(f"Failed to retrieve the app page. Status code: {response.status_code}")
        return None

# 앱 이름과 예상된 정확한 제목 매핑 리스트
app_names = {
    "카카오톡": "KakaoTalk",
    "파파고": "Papago",
    "카카오맵": "KakaoMap",
    "네이버": "NAVER",
    "쿠팡": "Coupang"
}

# 각 앱의 URL 수집 및 정보 크롤링
for app_name, expected_name in app_names.items():
    app_url = get_best_match_app_url(app_name, expected_name)
    if app_url:
        app_info = get_app_info(app_url)
        print(app_info)
    else:
        print(f"No URL found for app: {app_name}")


{'url': 'https://play.google.com/store/apps/details?id=com.kakao.talk', 'title': 'KakaoTalk : Messenger', 'description': 'KakaoTalk - fast, fun, and reliable messenger for all', 'category': '#1 popular communication'}
{'url': 'https://play.google.com/store/apps/details?id=com.naver.labs.translator', 'title': 'Naver Papago - AI Translator', 'description': 'Smart AI Translator, dreaming of a world communicating without language barriers', 'category': '#2 popular tools'}
No URL found for app: 카카오맵
{'url': 'https://play.google.com/store/apps/details?id=com.nhn.android.nmap', 'title': 'NAVER Map, Navigation', 'description': "South Korea's GPS navigation Get started right away", 'category': '#2 popular maps & navigation'}
{'url': 'https://play.google.com/store/apps/details?id=com.coupang.mobile.play', 'title': 'Coupang Play', 'description': "Enjoy COUPANG PLAY's Unlimited Movies&TV shows only for Coupang WOW members.", 'category': '#2 popular entertainment'}


In [13]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import quote

def get_search_url(app_name):
    base_url = "https://play.google.com/store/search?q="
    query = quote(app_name)
    return f"{base_url}{query}&c=apps"

def get_best_match_app_url(search_query, expected_name, num_results=5):
    search_url = get_search_url(search_query)
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    response = requests.get(search_url, headers=headers)
    
    if response.status_code == 200:
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')
        
        app_urls = []
        app_titles = []
        for a in soup.find_all('a', {'class': 'Si6A0c Gy4nib'}, limit=num_results):
            app_url = 'https://play.google.com' + a['href']
            app_title_div = a.find('div', {'class': 'WsMG1c nnK0zc'})
            if app_title_div:
                app_title = app_title_div.text
            else:
                app_title_span = a.find('span', {'class': 'DdYX5'})
                app_title = app_title_span.text if app_title_span else 'Title not found'
            app_urls.append(app_url)
            app_titles.append(app_title)
        
        # Find the best match
        best_match_url = None
        for i in range(len(app_titles)):
            if expected_name.lower() in app_titles[i].lower():
                best_match_url = app_urls[i]
                break
        
        return best_match_url
    else:
        print(f"Failed to retrieve the search page. Status code: {response.status_code}")
        return None

def get_app_info(app_url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    response = requests.get(app_url, headers=headers)
    
    if response.status_code == 200:
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')
        
        # 앱 제목 가져오기
        app_title = soup.find('h1', {'class': 'Fd93Bb'}).text if soup.find('h1', {'class': 'Fd93Bb'}) else 'Title not found'
        
        # 앱 설명 가져오기
        app_description_meta = soup.find('meta', {'itemprop': 'description'})
        app_description = app_description_meta['content'] if app_description_meta else 'Description not found'
        
        # 앱 카테고리 가져오기
        app_category_button = soup.find('a', {'itemprop': 'genre'})
        app_category = app_category_button.text if app_category_button else 'Category not found'
        
        return {
            'url': app_url,
            'title': app_title,
            'description': app_description,
            'category': app_category
        }
    else:
        print(f"Failed to retrieve the app page. Status code: {response.status_code}")
        return None

# 앱 이름과 예상된 정확한 제목 매핑 리스트
app_names = {
    "카카오톡": "KakaoTalk",
    "파파고": "Papago",
    "카카오맵": "KakaoMap",
    "네이버": "NAVER",
    "쿠팡": "Coupang"
}

# 각 앱의 URL 수집 및 정보 크롤링
for app_name, expected_name in app_names.items():
    app_url = get_best_match_app_url(app_name, expected_name)
    if app_url:
        app_info = get_app_info(app_url)
        print(app_info)
    else:
        print(f"No URL found for app: {app_name}")


{'url': 'https://play.google.com/store/apps/details?id=com.kakao.talk', 'title': 'KakaoTalk : Messenger', 'description': 'KakaoTalk - fast, fun, and reliable messenger for all', 'category': 'Category not found'}
{'url': 'https://play.google.com/store/apps/details?id=com.naver.labs.translator', 'title': 'Naver Papago - AI Translator', 'description': 'Smart AI Translator, dreaming of a world communicating without language barriers', 'category': 'Category not found'}
No URL found for app: 카카오맵
{'url': 'https://play.google.com/store/apps/details?id=com.nhn.android.nmap', 'title': 'NAVER Map, Navigation', 'description': "South Korea's GPS navigation Get started right away", 'category': 'Category not found'}
{'url': 'https://play.google.com/store/apps/details?id=com.coupang.mobile.play', 'title': 'Coupang Play', 'description': "Enjoy COUPANG PLAY's Unlimited Movies&TV shows only for Coupang WOW members.", 'category': 'Category not found'}


In [14]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import quote

def get_search_url(app_name):
    base_url = "https://play.google.com/store/search?q="
    query = quote(app_name)
    return f"{base_url}{query}&c=apps"

def get_best_match_app_url(search_query, expected_name, num_results=10):
    search_url = get_search_url(search_query)
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    response = requests.get(search_url, headers=headers)
    
    if response.status_code == 200:
        response.encoding = 'utf-8'
        html = response.content
        soup = BeautifulSoup(html, 'html.parser', from_encoding='utf-8')
        
        app_urls = []
        app_titles = []
        app_descriptions = []
        
        for a in soup.find_all('a', {'class': 'Si6A0c Gy4nib'}, limit=num_results):
            app_url = 'https://play.google.com' + a['href']
            app_title_div = a.find('div', {'class': 'WsMG1c nnK0zc'})
            if app_title_div:
                app_title = app_title_div.text
                app_urls.append(app_url)
                app_titles.append(app_title)
            
            app_description_div = a.find('div', {'class': 'b8cIId ReQCgd Q9MA7b'})
            if app_description_div:
                app_description = app_description_div.text
                app_descriptions.append(app_description)
        
        best_match_url = None
        best_match_description = None
        for i in range(len(app_titles)):
            if expected_name.lower() in app_titles[i].lower():
                best_match_url = app_urls[i]
                best_match_description = app_descriptions[i] if i < len(app_descriptions) else 'Description not found'
                break
        
        return best_match_url, best_match_description
    else:
        print(f"Failed to retrieve the search page. Status code: {response.status_code}")
        return None, None

def get_app_info(app_url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    response = requests.get(app_url, headers=headers)
    
    if response.status_code == 200:
        response.encoding = 'utf-8'
        html = response.content
        soup = BeautifulSoup(html, 'html.parser', from_encoding='utf-8')
        
        app_title = soup.find('h1', {'class': 'Fd93Bb'}).text if soup.find('h1', {'class': 'Fd93Bb'}) else 'Title not found'
        
        app_description_meta = soup.find('meta', {'itemprop': 'description'})
        app_description = app_description_meta['content'] if app_description_meta else 'Description not found'
        
        app_category_button = soup.find('a', {'itemprop': 'genre'})
        app_category = app_category_button.text if app_category_button else 'Category not found'
        
        return {
            'url': app_url,
            'title': app_title,
            'description': app_description,
            'category': app_category
        }
    else:
        print(f"Failed to retrieve the app page. Status code: {response.status_code}")
        return None

app_names = {
    "카카오톡": "KakaoTalk",
    "파파고": "Papago",
    "카카오맵": "KakaoMap",
    "네이버": "NAVER",
    "쿠팡": "Coupang"
}

for app_name, expected_name in app_names.items():
    app_url, app_description = get_best_match_app_url(app_name, expected_name)
    if app_url:
        app_info = get_app_info(app_url)
        app_info['description'] = app_description  # 검색 결과에서 가져온 설명을 사용
        print(app_info)
    else:
        print(f"No URL found for app: {app_name}")


No URL found for app: 카카오톡
No URL found for app: 파파고
No URL found for app: 카카오맵
No URL found for app: 네이버
No URL found for app: 쿠팡


In [15]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import quote

def get_search_url(app_name):
    base_url = "https://play.google.com/store/search?q="
    query = quote(app_name)
    return f"{base_url}{query}&c=apps"

def get_best_match_app_url(search_query, expected_name, num_results=10):
    search_url = get_search_url(search_query)
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    response = requests.get(search_url, headers=headers)
    
    if response.status_code == 200:
        response.encoding = 'utf-8'
        html = response.content
        soup = BeautifulSoup(html, 'html.parser', from_encoding='utf-8')
        
        app_urls = []
        app_titles = []
        app_descriptions = []
        
        for div in soup.find_all('div', {'class': 'Vpfmgd'}, limit=num_results):
            a_tag = div.find('a', {'class': 'Qfxief'})
            if a_tag:
                app_url = 'https://play.google.com' + a_tag['href']
                app_title_div = div.find('div', {'class': 'vWM94c'})
                if app_title_div:
                    app_title = app_title_div.text
                    app_urls.append(app_url)
                    app_titles.append(app_title)
                
                app_description_div = div.find('div', {'class': 'omXQ6c'})
                if app_description_div:
                    app_description = app_description_div.text
                    app_descriptions.append(app_description)
        
        best_match_url = None
        best_match_description = None
        for i in range(len(app_titles)):
            if expected_name.lower() in app_titles[i].lower():
                best_match_url = app_urls[i]
                best_match_description = app_descriptions[i] if i < len(app_descriptions) else 'Description not found'
                break
        
        return best_match_url, best_match_description
    else:
        print(f"Failed to retrieve the search page. Status code: {response.status_code}")
        return None, None

def get_app_info(app_url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    response = requests.get(app_url, headers=headers)
    
    if response.status_code == 200:
        response.encoding = 'utf-8'
        html = response.content
        soup = BeautifulSoup(html, 'html.parser', from_encoding='utf-8')
        
        app_title = soup.find('h1', {'class': 'Fd93Bb'}).text if soup.find('h1', {'class': 'Fd93Bb'}) else 'Title not found'
        
        app_description_meta = soup.find('meta', {'itemprop': 'description'})
        app_description = app_description_meta['content'] if app_description_meta else 'Description not found'
        
        app_category_button = soup.find('a', {'itemprop': 'genre'})
        app_category = app_category_button.text if app_category_button else 'Category not found'
        
        return {
            'url': app_url,
            'title': app_title,
            'description': app_description,
            'category': app_category
        }
    else:
        print(f"Failed to retrieve the app page. Status code: {response.status_code}")
        return None

# 앱 이름과 예상된 정확한 제목 매핑 리스트
app_names = {
    "카카오톡": "KakaoTalk",
    "파파고": "Papago",
    "카카오맵": "KakaoMap",
    "네이버": "NAVER",
    "쿠팡": "Coupang"
}

# 각 앱의 URL 수집 및 정보 크롤링
for app_name, expected_name in app_names.items():
    app_url, app_description = get_best_match_app_url(app_name, expected_name)
    if app_url:
        app_info = get_app_info(app_url)
        app_info['description'] = app_description  # 검색 결과에서 가져온 설명을 사용
        print(app_info)
    else:
        print(f"No URL found for app: {app_name}")


No URL found for app: 카카오톡
No URL found for app: 파파고
No URL found for app: 카카오맵
No URL found for app: 네이버
No URL found for app: 쿠팡
